In [23]:
%cd /home/ryanchao2012/projects/hack104-rec
%env SPARK_HOME=/opt/spark


from pyspark.sql import functions as f


/home/ryanchao2012/projects/hack104-rec
env: SPARK_HOME=/opt/spark


In [ ]:
import findspark
findspark.init()

import pyspark
spark_conf = pyspark.SparkConf()
spark_conf.setAll([
    ('spark.driver.memory', '10g'),
    ('spark.executor.memory', '10g'),
    ('spark.driver.maxResultSize', '10g')
])
spark = pyspark.sql.SparkSession.builder.config(conf=spark_conf).master('local[16]').getOrCreate()


from pyspark.sql import functions as f

display(spark)


# Job

In [2]:
!head -n1 data/job.json | jq --sort-keys

{
  "addr_no": 6001005004,
  "custno": "7c01228e-5bc6-479d-9b18-c95e6678a468",
  "description": "※輔導國小課業內容。\r\n※中、英語能力尤佳。\r\n需有相關課輔經驗者，無經驗者勿試。",
  "edu": 56,
  "exp_jobcat1": "0",
  "exp_jobcat2": "0",
  "exp_jobcat3": "0",
  "industry": "1005001009",
  "job": "國小安親課輔老師",
  "jobcat1": "2016002008",
  "jobcat2": "2016002011",
  "jobcat3": "0",
  "jobno": "4371531",
  "language1": 14444,
  "language2": 1111,
  "language3": 1111,
  "major_cat": "0",
  "major_cat2": "0",
  "major_cat3": "0",
  "need_emp": 1,
  "need_emp1": 2,
  "others": "1.提供勞健保\n2.待遇優\n3.需有耐心.愛心.抗壓性高/配合度佳-無誠者勿試/有安親經歷者尤佳",
  "period": 2,
  "role": 1,
  "role_status": 2049,
  "s2": 4,
  "s3": 0,
  "s9": 1,
  "salary_high": 990,
  "salary_low": 990,
  "startby": 2,
  "worktime": "週休二日"
}


In [35]:
# edu: Long -> Int
# exp_jobcat#: Str -> Long  (drop?)
# industry: Str -> Long
# jobcat#: Str -> Long  (drop?)
# jobno: Str -> Int
# language#: Long -> Int
# major_cat#: Str -> Long
# need_emp#: Long -> Int
# period: Long -> Int
# role: Long -> Int
# role_status: Long -> Int
# s2, s3, s9: Long -> Int
# salary_high/low: Long -> Int
# startby: Long -> Int

In [12]:
from hack104_rec.job import JobProcessed

In [3]:
%%time

JobProcessed.populate()

CPU times: user 79.4 ms, sys: 5.41 ms, total: 84.8 ms
Wall time: 1min 47s


In [15]:
type(job_sdf)

pyspark.sql.dataframe.DataFrame

In [13]:
job_sdf = JobProcessed.query()
job_sdf.printSchema()

root
 |-- custno: string (nullable = true)
 |-- jobno: integer (nullable = true)
 |-- description: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- job: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- others: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- addr_no: long (nullable = true)
 |-- edu: long (nullable = true)
 |-- exp_jobcat1: long (nullable = true)
 |-- exp_jobcat2: long (nullable = true)
 |-- exp_jobcat3: long (nullable = true)
 |-- industry: long (nullable = true)
 |-- jobcat1: long (nullable = true)
 |-- jobcat2: long (nullable = true)
 |-- jobcat3: long (nullable = true)
 |-- language1: long (nullable = true)
 |-- lan

In [7]:
job_sdf.select('*').show(1, vertical=True)

-RECORD 0---------------------------
 custno      | b1049129-a9f5-4b5... 
 jobno       | 4867765              
 description | [設計部主管
擅長集合住宅開發、... 
 job         | [專案建築師, [专案, 建筑, ... 
 others      | [具事務所或相關設計行業管理經驗、... 
 addr_no     | 6001001003           
 edu         | 56                   
 exp_jobcat1 | 2012003002           
 exp_jobcat2 | 2012001002           
 exp_jobcat3 | 0                    
 industry    | 1011003003           
 jobcat1     | 2012001002           
 jobcat2     | 2012003002           
 jobcat3     | 0                    
 language1   | 18888                
 language2   | 1111                 
 language3   | 1111                 
 major_cat   | null                 
 major_cat2  | 0                    
 major_cat3  | 0                    
 need_emp    | 1                    
 need_emp1   | 2                    
 period      | 8                    
 role        | 1                    
 role_status | 1                    
 s2          | 3                    
 s

# Company

In [5]:
!head -n1 data/company.json | jq --sort-keys

{
  "custno": "6b8867b8-beb2-4b5c-b3ff-6797a75205ff",
  "invoice": 10144499,
  "name": "屏東縣私立上林美語文理短期補習班",
  "product": "本補習班以安親和英文為主要的服務項目",
  "profile": "本補習班成立於民國96年,加盟佳音英語欲提供給地方上一個學習英語的良好環境,所以我們積極的培訓新老師,希望新老師能加入我們的行列",
  "welfare": "勞保，健保，員工制服，教育訓練，業績獎金，彈性輪休"
}


In [5]:
from hack104_rec.company import CompanyProcessed

In [3]:
%%time

CompanyProcessed.populate()

CPU times: user 41.8 ms, sys: 23.9 ms, total: 65.7 ms
Wall time: 2min 35s


In [6]:
company_sdf = CompanyProcessed.query()
company_sdf.printSchema()

root
 |-- custno: string (nullable = true)
 |-- invoice: long (nullable = true)
 |-- management: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- name: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- product: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- profile: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- welfare: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)



# Join Company and Job

In [7]:
# joined_sdf = job_sdf.join(company_sdf, 'custno', how='left')

In [3]:
joined_sdf = spark.read.parquet('data/job-company-joined.pq')

In [4]:
joined_sdf.printSchema()

root
 |-- custno: string (nullable = true)
 |-- jobno: integer (nullable = true)
 |-- description: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- job: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- others: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- token: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- addr_no: long (nullable = true)
 |-- edu: long (nullable = true)
 |-- exp_jobcat1: long (nullable = true)
 |-- exp_jobcat2: long (nullable = true)
 |-- exp_jobcat3: long (nullable = true)
 |-- industry: long (nullable = true)
 |-- jobcat1: long (nullable = true)
 |-- jobcat2: long (nullable = true)
 |-- jobcat3: long (nullable = true)
 |-- language1: long (nullable = true)
 |-- lan

In [7]:
%%time

joined_sdf.write.json('data/job-company-joined.json', mode='overwrite')

In [4]:
!http get localhost:9201/_cat/count/hack104\?v

HTTP/1.1 200 OK
content-encoding: gzip
content-length: 78
content-type: text/plain; charset=UTF-8

epoch      timestamp count
1531486706 12:58:26  458640



In [13]:
joined_sdf.count()

458638

In [11]:
rows[0]

Row(custno='005fd5fa-e11b-4c63-8b1a-4b77a6a5c242', jobno=7685413, description=Row(keyword='教授兒童美語，輔導美語課業，與家長保持聯繫。', token=['教授', '儿童', '美语', ',', '辅导', '美语', '课业', ',', '与', '家长', '保持', '联系', '保持联系', '。']), job=Row(keyword='兒童美語老師', token=['儿童', '美语', '老师']), others=Row(keyword=None, token=None), addr_no=6001002003, edu=24, exp_jobcat1=0, exp_jobcat2=0, exp_jobcat3=0, industry=1005001009, jobcat1=2016002011, jobcat2=0, jobcat3=0, language1=12222, language2=1111, language3=1111, major_cat=None, major_cat2=0, major_cat3=0, need_emp=0, need_emp1=0, period=-1, role=2, role_status=111, s2=0, s3=0, s9=2, salary_high=1, salary_low=1, startby=1, worktime=0, invoice=87792660, management=None, name=Row(keyword='學一文理美語短期補習班', token=['学一', '文理', '美语', '短期', '补习', '补习班']), product=Row(keyword='升大學、升高中文理補習班、兒童美語班  國小數學班', token=['升', '大学', '、', '升高', '中', '文理', '补习', '补习班', '、', '儿童', '美语', '班', '国', '小', '数学', '班']), profile=Row(keyword='本班經教育局立案，經營優良之文理補習班，歡迎具有教育熱忱  、積極負責之人士，加入我們的行列。詳細內容面試', token

In [7]:
joined_sdf.take(3)

[Row(custno='005fd5fa-e11b-4c63-8b1a-4b77a6a5c242', jobno=7685413, description=Row(keyword='教授兒童美語，輔導美語課業，與家長保持聯繫。', token=['教授', '儿童', '美语', ',', '辅导', '美语', '课业', ',', '与', '家长', '保持', '联系', '保持联系', '。']), job=Row(keyword='兒童美語老師', token=['儿童', '美语', '老师']), others=Row(keyword=None, token=None), addr_no=6001002003, edu=24, exp_jobcat1=0, exp_jobcat2=0, exp_jobcat3=0, industry=1005001009, jobcat1=2016002011, jobcat2=0, jobcat3=0, language1=12222, language2=1111, language3=1111, major_cat=None, major_cat2=0, major_cat3=0, need_emp=0, need_emp1=0, period=-1, role=2, role_status=111, s2=0, s3=0, s9=2, salary_high=1, salary_low=1, startby=1, worktime=0, invoice=87792660, management=None, name=Row(keyword='學一文理美語短期補習班', token=['学一', '文理', '美语', '短期', '补习', '补习班']), product=Row(keyword='升大學、升高中文理補習班、兒童美語班  國小數學班', token=['升', '大学', '、', '升高', '中', '文理', '补习', '补习班', '、', '儿童', '美语', '班', '国', '小', '数学', '班']), profile=Row(keyword='本班經教育局立案，經營優良之文理補習班，歡迎具有教育熱忱  、積極負責之人士，加入我們的行列。詳細內容面試', toke

In [10]:
%%time

joined_sdf.write.parquet('data/job-company-joined.pq', mode='overwrite')


# Dump to ES 

In [4]:
from hack104_rec.es.insert import bulk_insert, RowPreprocessor

In [5]:
%%time

# XXX: Not working, why??? 
bulk_insert(joined_sdf.limit(500).toLocalIterator(), preprocessor=RowPreprocessor())

Start preprocessing
Preprocessing finished, elapsed time: 2.283 sec.
Start `parallel_bulk` insert to es
Insert finished, 500 docs inserted, elapsed time: 0.508 sec.
CPU times: user 426 ms, sys: 67.2 ms, total: 493 ms
Wall time: 3.61 s


# Train Click

In [10]:
!head -n1 data/train-click.json | jq

{
  "jobno": "5314432",
  "date": "1527330445000",
  "action": "clickJob",
  "source": "web",
  "joblist": [
    "5314432",
    "3494303",
    "3859882",
    "4439691",
    "7416542",
    "7484247",
    "9063545",
    "8318922",
    "9431734",
    "9127494",
    "9904656",
    "9926223",
    "9950320",
    "10079727",
    "10378122",
    "6830159",
    "6856925",
    "10343328",
    "9730978",
    "8510715"
  ],
  "querystring": "ro=0&jobcat=2005003003%2C2005003002%2C2005003005&kwop=7&keyword=%E6%97%A5%E6%96%87&area=6001001000%2C6001005000&order=1&asc=0&mode=s&jobsource=n104bank1"
}


In [2]:
from hack104_rec import TrainClickProcessed, TrainClickExploded, TrainClickGrouped, TrainClickCTR

### Processed

In [3]:
%%time

TrainClickProcessed.populate()

CPU times: user 41.2 ms, sys: 29.1 ms, total: 70.3 ms
Wall time: 29 s


In [3]:
train_click_processed = TrainClickProcessed.query()

In [4]:
train_click_processed.printSchema()

root
 |-- action: string (nullable = true)
 |-- date: date (nullable = true)
 |-- joblist: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- jobno: long (nullable = true)
 |-- querystring: string (nullable = true)
 |-- source: string (nullable = true)
 |-- id: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- query_params: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- sctp: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- wktm: integer (nullable = true)
 |    |-- area: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- dep: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- edu: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- expcat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- incat: array (nullable = true)
 |    |    |-- el

In [5]:
train_click_processed.select('query_params.keyword', 'tokens').show()

+-------+-----------------+
|keyword|           tokens|
+-------+-----------------+
| bureau|         [bureau]|
|    台積 |             [台积]|
|     早班|             [早班]|
|     秘書|             [秘书]|
|     課級|             [课级]|
|軟體測試工程師|[软体, 测试, 工程, 工程师]|
|  外勤 維修|         [外勤, 维修]|
|     貿易|             [贸易]|
|   無經驗可|       [无, 经验, 可]|
|     馬達|             [马达]|
|  聯結車司機|      [联结, 车, 司机]|
|    Smt|            [Smt]|
|     電控|             [电控]|
|   行政工作|         [行政, 工作]|
|     電腦|             [电脑]|
|   冷凍食品|         [冷冻, 食品]|
|      農|              [农]|
|    SMT|            [SMT]|
|     法務|             [法务]|
|     成衣|             [成衣]|
+-------+-----------------+
only showing top 20 rows



### Grouped

In [10]:
from hack104_rec.train_click import get_tfidf

In [15]:
sdf = train_click_processed.limit(10).withColumn('text_score', get_tfidf.udf('joblist', 'tokens'))

In [34]:
sdf.printSchema()

root
 |-- action: string (nullable = true)
 |-- date: date (nullable = true)
 |-- joblist: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- jobno: long (nullable = true)
 |-- querystring: string (nullable = true)
 |-- source: string (nullable = true)
 |-- id: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- query_params: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- sctp: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- wktm: integer (nullable = true)
 |    |-- area: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- dep: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- edu: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- expcat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- incat: array (nullable = true)
 |    |    |-- el

In [53]:
(sdf.limit(2)
    .select('id', f.posexplode('text_score').alias('tscore_pos', 'text_score'))
    .select('id', 'text_score.*')
    .show()
)
sdf.

+-----------+--------+-----------+---+------+----------+----+-------+---------+-------+
|         id|   jobno|description|job|others|management|name|product|  profile|welfare|
+-----------+--------+-----------+---+------+----------+----+-------+---------+-------+
|34359738368| 9616675|   20.53792|0.0|   0.0|       0.0| 0.0|    0.0|      0.0|    0.0|
|34359738368| 6748608|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|      0.0|    0.0|
|34359738368| 9888331|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|17.117153|    0.0|
|34359738368| 9888332|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|18.432102|    0.0|
|34359738368| 9888351|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|      0.0|    0.0|
|34359738368| 4190009|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|17.117153|    0.0|
|34359738368| 4793570|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|      0.0|    0.0|
|34359738368| 4823347|        0.0|0.0|   0.0|       0.0| 0.0|    0.0|18.068455|    0.0|
|34359738368| 6345406|        0.

In [51]:
(sdf.limit(2)
    .select('id', f.posexplode('joblist').alias('jobpos', 'joblist'))
#     .select('*', f.posexplode('text_score').alias('tscore_pos', 'text_score'))
#     .select('joblist', 'text_score.*')
    .show()
)

+-----------+------+--------+
|         id|jobpos| joblist|
+-----------+------+--------+
|34359738368|     0| 9616675|
|34359738368|     1| 6748608|
|34359738368|     2| 9888331|
|34359738368|     3| 9888332|
|34359738368|     4| 9888351|
|34359738368|     5| 4190009|
|34359738368|     6| 4793570|
|34359738368|     7| 4823347|
|34359738368|     8| 6345406|
|34359738368|     9| 6476231|
|34359738368|    10| 8735015|
|34359738368|    11| 9693438|
|34359738368|    12| 9474016|
|34359738368|    13|10353576|
|34359738368|    14|10166493|
|34359738368|    15|10128477|
|34359738368|    16|10042661|
|34359738368|    17| 6467598|
|34359738368|    18| 7262484|
|34359738368|    19| 7766505|
+-----------+------+--------+
only showing top 20 rows



In [17]:
%%time

TrainClickGrouped.populate()

CPU times: user 35.3 ms, sys: 19.5 ms, total: 54.8 ms
Wall time: 19.5 s


In [3]:
train_click_grouped = TrainClickGrouped.query()

AnalysisException: 'Unable to infer schema for Parquet. It must be specified manually.;'

In [5]:
train_click_grouped.printSchema()

root
 |-- source: string (nullable = true)
 |-- date: date (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- query_params: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- sctp: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- wktm: integer (nullable = true)
 |    |-- area: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- dep: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- edu: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- expcat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- incat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- isnew: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- jobcat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |  

### Exploded

In [18]:
%%time 

TrainClickExploded.populate()

CPU times: user 11.7 ms, sys: 1.32 ms, total: 13 ms
Wall time: 13.2 s


In [3]:
train_click_exploded = TrainClickExploded.query()

In [20]:
train_click_exploded.printSchema()

root
 |-- source: string (nullable = true)
 |-- date: date (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- query_params: struct (nullable = true)
 |    |-- keyword: string (nullable = true)
 |    |-- sctp: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- wktm: integer (nullable = true)
 |    |-- area: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- dep: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- edu: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- expcat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- incat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- isnew: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |    |-- jobcat: array (nullable = true)
 |    |    |-- element: integer (containsNull = true)
 |  

In [22]:
train_click_exploded.select('query_params.*').show(n=1, vertical=True)

-RECORD 0---------
 keyword  | 屈臣氏   
 sctp     | 0     
 mode     | 1     
 wktm     | 0     
 area     | []    
 dep      | []    
 edu      | []    
 expcat   | []    
 incat    | []    
 isnew    | []    
 jobcat   | []    
 cat      | []    
 jobexp   | []    
 ro       | [1]   
 rostatus | []    
 s9       | []    
 wf       | []    
 wt       | []    
 zone     | []    
 order    | 1     
 asc      | 0     
 scmax    | -1    
 scmin    | -1    
 kwop     | false 
 m        | false 
 s5       | null  
 sr       | null  
only showing top 1 row



### CTR

In [28]:
%%time

TrainClickCTR.populate()

AnalysisException: "cannot resolve '`action`' given input columns: [rel, job, date, pos_in_list, click, datetime, source, gid, query_params];;\n'Aggregate [date#5911, source#5910, 'action, query_params#5913.keyword, 'job_in_list], [date#5911, source#5910, 'action, query_params#5913.keyword AS keyword#5950, 'job_in_list, count(1) AS impr#5946L, sum(click#5926L) AS click#5948L]\n+- AnalysisBarrier\n      +- Project [source#5910, date#5911, datetime#5912, query_params#5913, gid#5914L, pos_in_list#5915, job#5916L, rel#5917L, rel#5917L AS click#5926L]\n         +- Relation[source#5910,date#5911,datetime#5912,query_params#5913,gid#5914L,pos_in_list#5915,job#5916L,rel#5917L] parquet\n"